### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_random

CONCEPT_NAME = concepts.concept_random.__name__


In [12]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [14]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2
    alpha = 0.2 # 0 = argmax, 1 = distribution

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state, alpha=alpha)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 2500 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   2%|▏         | 62/2500 [00:08<05:20,  7.61it/s]
Positive cases: 2541it [02:13, 13.93it/s]                          

In [15]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2500, 4, 5, 5)


In [ ]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [9]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  29
Duplicate count in negative:  323


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [17]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [18]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.8585 - val_loss: 0.8423
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8339 - val_loss: 0.8263
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8230 - val_loss: 0.8187
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8171 - val_loss: 0.8138
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8127 - val_loss: 0.8097
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8088 - val_loss: 0.8060
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8053 - val_loss: 0.8025
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8018 - val_loss: 0.7992
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7986 - val_loss: 0.7961
Epoch 10/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7954 - val_loss: 0.7930

In [19]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 276.04it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9825 - val_loss: 0.9573
Epoch 2/50
125/125 [==============================] - 0s 983us/step - loss: 0.9338 - val_loss: 0.9106
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8891 - val_loss: 0.8677
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8488 - val_loss: 0.8295
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8134 - val_loss: 0.7967
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7833 - val_loss: 0.7689
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7583 - val_loss: 0.7465
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7378 - val_loss: 0.7281
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7214 - val_loss: 0.7146
Epoch 10/50
125/125 [==============================] - 0s 1ms/st

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 288.23it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0100 - val_loss: 0.9795
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.9747 - val_loss: 0.9502
Epoch 3/50
125/125 [==============================] - 0s 972us/step - loss: 0.9453 - val_loss: 0.9254
Epoch 4/50
125/125 [==============================] - 0s 994us/step - loss: 0.9196 - val_loss: 0.9024
Epoch 5/50
125/125 [==============================] - 0s 959us/step - loss: 0.8960 - val_loss: 0.8799
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8742 - val_loss: 0.8605
Epoch 7/50
125/125 [==============================] - 0s 955us/step - loss: 0.8538 - val_loss: 0.8412
Epoch 8/50
125/125 [==============================] - 0s 976us/step - loss: 0.8350 - val_loss: 0.8254
Epoch 9/50
125/125 [==============================] - 0s 964us/step - loss: 0.8178 - val_loss: 0.8099
Epoch 10/50
125/125 [==============================] -

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 269.63it/s]


...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0373 - val_loss: 0.9947
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.9931 - val_loss: 0.9681
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.9653 - val_loss: 0.9445
Epoch 4/50
125/125 [==============================] - 0s 968us/step - loss: 0.9406 - val_loss: 0.9240
Epoch 5/50
125/125 [==============================] - 0s 1000us/step - loss: 0.9185 - val_loss: 0.9040
Epoch 6/50
125/125 [==============================] - 0s 971us/step - loss: 0.8975 - val_loss: 0.8856
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8781 - val_loss: 0.8692
Epoch 8/50
125/125 [==============================] - 0s 979us/step - loss: 0.8603 - val_loss: 0.8520
Epoch 9/50
125/125 [==============================] - 0s 999us/step - loss: 0.8435 - val_loss: 0.8372
Epoch 10/50
125/125 [==============================] - 

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 288.66it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0195 - val_loss: 0.9956
Epoch 2/50
125/125 [==============================] - 0s 993us/step - loss: 0.9876 - val_loss: 0.9716
Epoch 3/50
125/125 [==============================] - 0s 989us/step - loss: 0.9606 - val_loss: 0.9497
Epoch 4/50
125/125 [==============================] - 0s 990us/step - loss: 0.9364 - val_loss: 0.9289
Epoch 5/50
125/125 [==============================] - 0s 991us/step - loss: 0.9140 - val_loss: 0.9082
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8933 - val_loss: 0.8892
Epoch 7/50
125/125 [==============================] - 0s 975us/step - loss: 0.8740 - val_loss: 0.8713
Epoch 8/50
125/125 [==============================] - 0s 964us/step - loss: 0.8560 - val_loss: 0.8554
Epoch 9/50
125/125 [==============================] - 0s 995us/step - loss: 0.8396 - val_loss: 0.8396
Epoch 10/50
125/125 [==============================]

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 292.33it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0525 - val_loss: 1.0100
Epoch 2/50
125/125 [==============================] - 0s 980us/step - loss: 0.9930 - val_loss: 0.9834
Epoch 3/50
125/125 [==============================] - 0s 990us/step - loss: 0.9685 - val_loss: 0.9604
Epoch 4/50
125/125 [==============================] - 0s 980us/step - loss: 0.9468 - val_loss: 0.9396
Epoch 5/50
125/125 [==============================] - 0s 968us/step - loss: 0.9265 - val_loss: 0.9200
Epoch 6/50
125/125 [==============================] - 0s 983us/step - loss: 0.9075 - val_loss: 0.9022
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8898 - val_loss: 0.8847
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8728 - val_loss: 0.8683
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8569 - val_loss: 0.8532
Epoch 10/50
125/125 [==============================] - 0